
# 제조업 A/B 테스트 실습 (t-테스트 활용)

제조업 환경에서 A/B 테스트를 수행하는 방법을 단계별로 수행합니다. A/B 테스트는 두 그룹 간의 차이를 비교하기 위한 통계적 방법입니다. 여기서는 두 생산 라인의 불량률을 비교합니다.

## 1. 데이터 준비
먼저, 데이터 엑셀 파일을 불러옵니다.

In [ ]:
import pandas as pd

# 엑셀 파일에서 데이터를 읽어옵니다. (예제 파일: ab-test.xlsx)
test_cases = pd.read_excel("ab-test.xlsx", sheet_name=None)

# 데이터 확인
for case_name, data in test_cases.items():
    print(f"--- {case_name} ---")
    display(data.head())

## 2. 데이터 요약
각 생산 라인의 불량률을 계산합니다. 불량률은 총 생산량 대비 불량품의 비율입니다.

In [ ]:
# 각 케이스별 불량률 계산 함수
def calculate_defect_stats(data):
    mean_a = data['line_a'].mean()
    std_a = data['line_a'].std()
    mean_b = data['line_b'].mean()
    std_b = data['line_b'].std()
    return mean_a, std_a, mean_b, std_b

# 각 케이스에 대해 불량률 계산
defect_stats = {case: calculate_defect_stats(data) for case, data in test_cases.items()}

for case, stats in defect_stats.items():
    print(f"---<{case}>---")
    print(f"생산 라인 A의 불량률: 평균={stats[0]:.2%}, 표준편차={stats[1]:.2%}")
    print(f"생산 라인 B의 불량률: 평균={stats[2]:.2%}, 표준편차={stats[3]:.2%}")


## 3. t-테스트 수행
t-테스트를 사용하여 두 생산 라인의 불량률 차이가 통계적으로 유의미한지 검정합니다.

In [ ]:
# t-테스트를 사용하여 두 생산 라인의 불량률 차이가 통계적으로 유의미한지 검정합니다.
from scipy import stats

# 각 케이스별 t-테스트 수행 함수
def perform_t_test(data):
    t_stat, p_value = stats.ttest_ind(data['line_a'], data['line_b'])
    return t_stat, p_value

# 각 케이스에 대해 t-테스트 수행
t_test_results = {case: perform_t_test(data) for case, data in test_cases.items()}

for case, results in t_test_results.items():
    print(f"p-값: {results[1]:.4f}")

## 4. 결과 해석
유의수준을 설정하고, p-값을 기준으로 두 생산 라인의 불량률 차이가 유의미한지 판단합니다.

In [ ]:
# 유의수준 설정 (예: 0.05)
alpha = 0.05

for case, results in t_test_results.items():
    if results[1] < alpha:
        print(f"{case} - 두 생산 라인의 불량률 차이는 통계적으로 유의미합니다.")
    else:
        print(f"{case} - 두 생산 라인의 불량률 차이는 통계적으로 유의미하지 않습니다.")

## 5. 데이터 시각화
히스토그램을 사용하여 두 생산 라인의 불량률을 시각화합니다.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 각 케이스별 불량률 히스토그램 시각화 함수
def plot_defect_density(data, case_name):
    sns.kdeplot(data['line_a'], fill=True, label='Line A', bw_adjust=0.5)
    sns.kdeplot(data['line_b'], fill=True, label='Line B', bw_adjust=0.5)
    plt.legend(loc='upper right')
    plt.title(f'{case_name} - Defect Rate Density Comparison')
    plt.xlabel('Product Status (0: Normal, 1: Defective)')
    plt.ylabel('Density')
    plt.xlim(-0.02, 0.13)  # x축 범위 고정
    plt.show()

# 각 케이스에 대해 히스토그램 시각화
for case_name, data in test_cases.items():
    plot_defect_density(data, case_name)

# 베이시안 A/B 테스트 실습

베이시안 방식으로 A/B 테스트를 수행하는 방법을 단계별로 진행합니다.

## 1. 데이터 준비

In [ ]:
import pandas as pd

# 엑셀 파일에서 데이터를 읽어옵니다. (예제 파일: ab-test.xlsx)
test_cases = pd.read_excel("ab-test.xlsx", sheet_name=None)

## 2. 베이시안 A/B 테스트 수행

베이시안 접근법은 사전 확률(Prior Probability)과 관측 데이터를 결합하여 사후 확률(Posterior Probability)을 계산합니다. 사전 확률은 우리가 사전에 알고 있는 지식이나 믿음을 나타내며, 관측 데이터를 통해 이를 업데이트합니다.

In [ ]:
# !pip install pymc

아래 과정은 두 생산 라인의 불량률을 베이시안 접근 방식을 사용하여 비교합니다. 각 생산 라인의 불량률에 대한 사전 확률 분포를 설정하고, 관측된 데이터를 사용하여 사후 확률 분포를 계산합니다. 그 후, 두 생산 라인의 불량률 차이에 대한 사후 확률 분포를 분석하고 시각화합니다.

구체적인 단계는 다음과 같습니다:
1. 각 생산 라인의 불량품 수와 총 샘플 수를 계산합니다.
2. 베이시안 모델을 설정하여 각 생산 라인의 불량률에 대한 사전 확률 분포를 정의합니다.
3. 관측된 데이터를 기반으로 불량률에 대한 우도 함수를 정의합니다.
4. MCMC 방법을 사용하여 사후 확률 분포를 샘플링합니다.
5. 두 생산 라인의 불량률 차이에 대한 사후 확률 분포를 분석하고, B 라인이 A 라인보다 나을 확률을 계산합니다.
6. 결과를 시각화하여 불량률 차이에 대한 신뢰 구간과 확률 분포를 보여줍니다.

In [ ]:
import pymc as pm
import numpy as np
import arviz as az

def perform_bayesian_ab_test(data):
    defects_a = data['line_a'].sum()
    defects_b = data['line_b'].sum()
    total_a = len(data['line_a'])
    total_b = len(data['line_b'])

    with pm.Model() as model:
        # 사전 확률 정의
        p_a = pm.Beta('p_a', alpha=1, beta=1)
        p_b = pm.Beta('p_b', alpha=1, beta=1)
        
        # 우도 정의
        obs_a = pm.Binomial('obs_a', n=total_a, p=p_a, observed=defects_a)
        obs_b = pm.Binomial('obs_b', n=total_b, p=p_b, observed=defects_b)
        
        # 차이 계산
        diff = pm.Deterministic('diff', p_b - p_a)
        
        # 샘플링
        trace = pm.sample(2000, return_inferencedata=True)

    # 결과 분석
    diff_samples = trace.posterior['diff'].values.flatten()
    prob_b_better = np.mean(diff_samples < 0)

    print("\n베이시안 A/B 테스트 결과:")
    print(f"B가 A보다 나을 확률: {prob_b_better:.2%}")
    print(f"차이의 95% 신뢰구간: [{np.percentile(diff_samples, 2.5):.4f}, {np.percentile(diff_samples, 97.5):.4f}]")

    # 결과 시각화
    az.plot_posterior(trace, var_names=['diff'], ref_val=0)
    
    return trace

In [ ]:
perform_bayesian_ab_test(test_cases['case1'])

In [ ]:
perform_bayesian_ab_test(test_cases['case2'])

In [ ]:
perform_bayesian_ab_test(test_cases['case3'])